In [1]:
pip install alpha_vintage

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement alpha_vintage (from versions: none)
ERROR: No matching distribution found for alpha_vintage


In [2]:
pip install alpha_vantage


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


# 1. Define a Class ScriptData which can fetch US Stock data using Alpha Vantage.

In [4]:
from alpha_vantage.timeseries import TimeSeries

In [5]:
import pandas as pd

In [6]:
ALPHAVANTAGE_API_KEY = 'OC43FD8AED2GOMCN'

In [7]:
class ScriptData:
    def __init__(self):
        self.data = {}
    
    def __getitem__(self,key):
        return self.data[key]
    
    def __setitem__(self,key,value):
        self.data[key] = value
    
    def __contains__(self,key):
        return key in self.data
        
    
    def fetch_intraday_data(self,script):
        ts = TimeSeries(key=ALPHAVANTAGE_API_KEY,indexing_type='date')
        data,meta_data = ts.get_intraday(script)
        self[script] = data
        
    
    def convert_intraday_data(self,script):
        temp = self[script]
        df = pd.DataFrame(temp)
        df = df.transpose()
        df = df.reset_index()
        df = df.rename(columns={'index':'timestamp','1. open':'open',
                                '2. high':'high','3. low':'low',
                                '4. close':'close','5. volume':'volume'
                               })
        self[script] = df
        


In [8]:
script_data = ScriptData()

In [9]:
script_data.fetch_intraday_data('GOOGL')

In [10]:
script_data.convert_intraday_data('GOOGL')

In [11]:
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-06-12 19:45:00,123.9200,123.9200,123.8200,123.8300,5411
1,2023-06-12 19:30:00,123.8500,123.9800,123.8500,123.9500,8597
2,2023-06-12 19:15:00,123.9400,123.9400,123.7600,123.8500,1112
3,2023-06-12 19:00:00,123.9000,123.9000,123.8000,123.8000,953
4,2023-06-12 18:45:00,123.8200,123.9400,123.7900,123.9400,6317
...,...,...,...,...,...,...
95,2023-06-09 11:45:00,122.8600,122.8650,122.3201,122.3600,588981
96,2023-06-09 11:30:00,122.7200,123.0500,122.6100,122.8800,727964
97,2023-06-09 11:15:00,123.4600,123.4900,122.7000,122.7250,693256
98,2023-06-09 11:00:00,122.9050,123.4868,122.8750,123.4600,512181


In [12]:
'GOOGL' in script_data

True

In [13]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-06-12 19:45:00,181.8500,182.7000,181.5000,182.5099,148977
1,2023-06-12 19:30:00,183.8800,183.9900,181.6300,181.7700,209997
2,2023-06-12 19:15:00,184.2000,184.3500,183.7900,183.7900,74450
3,2023-06-12 19:00:00,184.2900,184.2900,184.1700,184.1700,8656
4,2023-06-12 18:45:00,184.0700,184.2800,184.0700,184.2800,17603
...,...,...,...,...,...,...
95,2023-06-09 12:00:00,180.9000,181.3900,180.6300,181.3700,1356637
96,2023-06-09 11:45:00,181.4550,181.4900,180.8600,180.9200,1156788
97,2023-06-09 11:30:00,181.2250,181.4900,180.9700,181.4587,1570584
98,2023-06-09 11:15:00,182.0600,182.0650,181.1300,181.2300,1941793


In [14]:
'GOOGL' in script_data

True

In [15]:
'AAPL' in script_data

True

In [16]:
'NVDA' in script_data

False

In [17]:
def indicator1(df,timeperiod):
    df['indicator'] = df['close'].rolling(timeperiod).mean()
    new_df = df[['timestamp','indicator']].copy()
    return new_df

In [18]:
indicator1(script_data['GOOGL'],timeperiod=5)

,timestamp,indicator
0,2023-06-12 19:45:00,NaN
1,2023-06-12 19:30:00,NaN
2,2023-06-12 19:15:00,NaN
3,2023-06-12 19:00:00,NaN
4,2023-06-12 18:45:00,123.874
...,...,...
95,2023-06-09 11:45:00,122.324
96,2023-06-09 11:30:00,122.412
97,2023-06-09 11:15:00,122.511
98,2023-06-09 11:00:00,122.757


In [19]:
class Strategy:
    def __init__(self,script_name):
        self.script = script_name
        self.data = {}
        self.indicator_data = None
    
    def get_script_data(self):
        script_data = ScriptData()
        script_data.fetch_intraday_data(self.script)
        script_data.convert_intraday_data(self.script)
        self.data = script_data[self.script]
        self.compute_indicator_data()
        
    def compute_indicator_data(self):
        self.indicator_data = indicator1(self.data,2)
    
    def get_signals(self):
        signals = pd.DataFrame({'timestamp':self.data['timestamp']})
        signals['signal'] = 'NO_SIGNAL'
        self.data['close'] = self.data['close'].astype(float)
        signals.loc[self.indicator_data['indicator'] > self.data['close'], 'signal'] = 'BUY'
        signals.loc[self.indicator_data['indicator'] < self.data['close'], 'signal'] = 'SELL'
        display(signals)
        

In [20]:
strategy = Strategy('NVDA')

In [21]:
strategy.get_script_data()

In [22]:
strategy.get_signals()

,timestamp,signal
0,2023-06-12 19:45:00,NO_SIGNAL
1,2023-06-12 19:30:00,BUY
2,2023-06-12 19:15:00,SELL
3,2023-06-12 19:00:00,SELL
4,2023-06-12 18:45:00,SELL
...,...,...
95,2023-06-09 11:45:00,BUY
96,2023-06-09 11:30:00,SELL
97,2023-06-09 11:15:00,BUY
98,2023-06-09 11:00:00,SELL


In [23]:
pip install pyalgotrading

Note: you may need to restart the kernel to use updated packages.


In [24]:
from pyalgotrading import LivePlot
import matplotlib.pyplot as plt

ImportError: cannot import name 'LivePlot' from 'pyalgotrading' (C:\Users\ACER\anaconda3\lib\site-packages\pyalgotrading\__init__.py)